In [1]:
import json
from typing import List, Literal, Optional

import tiktoken
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
import uuid


from MemoryManager import MemoryHandler
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank
import os

llm= ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
embed = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2",truncate="NONE",)
memory_manager=MemoryHandler(llm,embed)


In [2]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

def mem_routing_function(inputs):
    query=inputs["input"]
    config=inputs["config"]
    output=memory_manager.memory_routing(query, config)
    return output


def create_memory_items(inputs):
    query=inputs["input"]
    memory_items = memory_manager.query_to_memory_items(query=query)
    return memory_items

runnable_parallel_1 = RunnableLambda(mem_routing_function)
runnable_parallel_2 = RunnableLambda(create_memory_items)
    

def execute_memory_operations(inputs):
    mem_ops=inputs["mem_ops"]
    memory_items_for_saving=inputs["mem_items"]["facts"]
    if 'save_memory' in mem_ops.lower():        
        memories, ids=memory_manager.save_recall_memory(memory_items_for_saving, memory_manager.config)
        output = ids
    elif "update_memory" in mem_ops.lower():        
        memories, ids = memory_manager.save_recall_memory(memory_items_for_saving, memory_manager.config)
        output = ids
    elif "no operation":
        output=llm.invoke(query).content 
    return output

sequence = RunnablePassthrough() | {  # this dict is coerced to a RunnableParallel
    "mem_ops": runnable_parallel_1,
    "mem_items": runnable_parallel_2
    } | execute_memory_operations



## populating the memories via conversations 

In [3]:
config = {"configurable": {"user_id": "babe", "thread_id": "1"}}
memory_manager.config=config  

parallel_output = sequence.invoke(input={"input":"hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob.", "config":config})
parallel_output

['4f2aa2e8-df34-4ee9-9f63-8291cce666e9',
 '64703875-a10c-4714-ad26-8589931867bf',
 'abaa3db9-ebe4-4198-9d23-66694027aeeb',
 '1389022f-f35c-48e1-9e0d-87cdd189fd5a']

In [4]:
config = {"configurable": {"user_id": "babe", "thread_id": "2"}}
memory_manager.config=config  

parallel_output = sequence.invoke(input={"input":"I had a fight with Rob, he is no longer my best friend !", "config":config})
parallel_output

not implemented error


['75a9837a-ed1a-434e-9412-ade4c8d17c77',
 'f9496973-1ec9-4d80-9bf7-f9a0319ccede']

In [5]:
retriever = memory_manager.recall_vector_store.as_retriever(         search_type="similarity_score_threshold",
                                                            search_kwargs={'filter': {'user':'babe'}, {'score_threshold': 0.8}})



## wrap retriever into LLM

In [6]:
prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are assistant with ability to memorize conversations from the user. You should always answer user query based on the following context:\n<Documents>\n{context}\n</Documents>. \
                    Be polite and helpful.",
                ),
                ("user", "{input}"),
            ]
        )
retriever_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm 
)

In [8]:
out=retriever_chain.invoke("do you remember if I am friend with Rob?")
out.content

'I recall that you mentioned having a fight with Rob and also said that Rob is no longer your best friend. However, I also remember that you mentioned your best friend is a chicken named Rob. It seems there might be a bit of confusion or perhaps a change in your relationship with Rob.'